Bioinformatics Project 
PART-1  Extracting data from CHEMBL API and cleaning

In [ ]:
#installing necessary libraries and frameworks
! pip install chembl_webresource_client

In [3]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [5]:
# Target search for acetylcholinesterase target
target = new_client.target
target_query = target.search('acetylcholinesterase')
#the target_query returns list of dictionaries having entries of targets and its attributes 
#in next line of code changing the query data we got into dataframe
targets = pd.DataFrame.from_dict(target_query)
targets


,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Drosophila melanogaster,Acetylcholinesterase,18.0,False,CHEMBL2242744,"[{'accession': 'P07140', 'component_descriptio...",SINGLE PROTEIN,7227
1,[],Homo sapiens,Acetylcholinesterase,16.0,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606
2,[],Torpedo californica,Acetylcholinesterase,16.0,False,CHEMBL4780,"[{'accession': 'P04058', 'component_descriptio...",SINGLE PROTEIN,7787
3,[],Mus musculus,Acetylcholinesterase,16.0,False,CHEMBL3198,"[{'accession': 'P21836', 'component_descriptio...",SINGLE PROTEIN,10090
4,[],Rattus norvegicus,Acetylcholinesterase,16.0,False,CHEMBL3199,"[{'accession': 'P37136', 'component_descriptio...",SINGLE PROTEIN,10116
5,[],Electrophorus electricus,Acetylcholinesterase,16.0,False,CHEMBL4078,"[{'accession': 'O42275', 'component_descriptio...",SINGLE PROTEIN,8005
6,[],Bos taurus,Acetylcholinesterase,16.0,False,CHEMBL4768,"[{'accession': 'P23795', 'component_descriptio...",SINGLE PROTEIN,9913
7,[],Bemisia tabaci,AChE2,16.0,False,CHEMBL2366409,"[{'accession': 'B3SST5', 'component_descriptio...",SINGLE PROTEIN,7038
8,[],Leptinotarsa decemlineata,Acetylcholinesterase,16.0,False,CHEMBL2366490,"[{'accession': 'Q27677', 'component_descriptio...",SINGLE PROTEIN,7539
9,[],Nephotettix cincticeps,Ace-orthologous acetylcholinesterase,16.0,False,CHEMBL2366514,"[{'accession': 'Q9NJH6', 'component_descriptio...",SINGLE PROTEIN,94400


In [7]:
#lets know how many targets  we have got from query-- 30 rows and 9 columns

targets.shape

(30, 9)

In [ ]:
# lets select target at the entry 1
select_target = targets.target_chembl_id[1]
select_target

In [ ]:
#again approaching API to retrieve data from chembl server 
activity = new_client.activity
res = activity.filter(target_chembl_id=select_target).filter(standard_type="IC50")

In [ ]:
# since the entire dataset take time to load, trying to Retrieve  the first 3000 entries
limited_res = res[:3000]

# Convert the  results into a pandas DataFrame
df = pd.DataFrame.from_dict(limited_res)

# Displaying dataframe
df

In [ ]:
#capture the data and save in a activity_data.csv file
output_file = "activity_data.csv"
df.to_csv(output_file, index=False) 

In [ ]:
#loading the dataset
data = pd.read_csv("activity_data.csv")

In [ ]:
data.columns

In [ ]:
# extracting required fields from the activity_data.csv file
column_to_extract = ['molecule_chembl_id','target_organism','value']

In [ ]:
#creating a new data frame from  df
new_df = data[column_to_extract]
print(new_df)

In [ ]:
#creating a file with required columns
new_df.to_csv("activity_data_1",index =False)

In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
# Load your DataFrame (replace with your file)
df = pd.read_csv('activity_data_1.csv') 

# Function to fetch SMILES
def fetch_smiles(chembl_id):
    url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{chembl_id}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            # Parse XML
            root = ET.fromstring(response.content)
            # Extract SMILES
            smiles = root.find(".//canonical_smiles")  # Adjust XPath if needed
            return smiles.text if smiles is not None else None
        else:
            print(f"Error: Received status code {response.status_code} for {chembl_id}")
    except Exception as e:
        print(f"Exception for {chembl_id}: {e}")
    return None

df['SMILES'] = df['molecule_chembl_id'].apply(fetch_smiles)
# Save results
df.to_csv('activity_data_with_smiles1.csv', index=False)
